In [2]:
import glob
import numpy as np

import cv2

In [30]:
def mask_image(image: cv2.Mat):
    # mask using fillpoly
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    a = 300
    b = 100
    mask = cv2.fillPoly(
        mask,
        [
            np.array([[a, 100], [b, image.shape[0]], [image.shape[1] - b, image.shape[0]], [image.shape[1] - a, 100],])
        ],
        255,
    )
    return cv2.bitwise_and(image, image, mask=mask)

# load images as imread from dummy folder
images = glob.glob('dummy/*.png')
for image in images:
    frame = cv2.imread(image)
    
    # delete useless margins
    frame = mask_image(frame)
    # to gray scale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    cv2.imshow('original', gray)
    # threshold
    thresh = cv2.threshold(gray, 210, 255, cv2.THRESH_BINARY)[1]
    canny = cv2.Canny(thresh, 75, 200)
    cv2.imshow('canny', canny)
    # hough transform and draw lines
    lines = cv2.HoughLines(canny, 1, np.pi / 90, 100)
    if lines is None:
        continue

    # draw lines
    canny = cv2.cvtColor(canny, cv2.COLOR_GRAY2BGR)
    thresh_draw = canny.copy()
    for line in lines:
        rho, theta = *line[0],
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 10000 * (-b))
        y1 = int(y0 + 10000 * (a))
        x2 = int(x0 - 10000 * (-b))
        y2 = int(y0 - 10000 * (a))

        cv2.line(thresh_draw, (x1, y1), (x2, y2), (0, 255, 0), 1)

    # show with blended thresh and lines
    cv2.imshow('frame', cv2.addWeighted(canny, 0.2, thresh_draw, 0.8, 0))
    cv2.waitKey(0)
cv2.destroyAllWindows()